In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path

import plotly.express as px
import re

import plotly.io as pio
pio.renderers.default = "browser"

from tensorflow.keras.layers import (
    LSTM,
    Conv1D,
    Dense,
    Flatten,
    Input,
    MaxPooling1D,
    TimeDistributed,
)
from tensorflow.keras.models import Sequential

In [ ]:
def extract_class(filename):
    match = re.search(r'benchy_\d+_(.*?)\.parquet\.gzip', filename)
    if match:
        return match.group(1)
    return None

Printer Resolution: 0.1mm

Printer Speed: 60mm/s

Acquisition Sample rate: 3200 Hz

Minimum print steps: 2

In [ ]:
sample_rate = 3200
print_resolution = 0.1
print_speed = 60
minimum_print_steps = 2
min_print_window = minimum_print_steps*print_resolution/print_speed
samples_per_window = min_print_window*sample_rate
print(f"Minimum print window: {min_print_window}")
print(f"Samples per window: {samples_per_window}")

## Reproduction of Article model

In [ ]:
def window_using_rolling(df,window=44,step=22,):
    rolling = df[["accel_x", "accel_y", "accel_z", "accel_x"]].rolling(window=window,step=step)
    np.array(list(rolling)[2:])

In [ ]:
df_all = pd.DataFrame()
list_classes = set()
for file in Path("downsampled_200").glob("benchy_*"):
    list_classes.add(extract_class(file.name))
print(list_classes)

class_index = dict()
for i,class_name in enumerate(list_classes):
    class_index[class_name] = int(i)
print(class_index)

In [ ]:
for file in Path("downsampled_200").glob("benchy_*"):
    print(extract_class(file))
    df = pd.read_parquet(file)
    df_all = pd.concat([df_all,df], ignore_index=True)

In [174]:
df = pd.read_parquet(file)
rolling = df[["accel_x", "accel_y", "accel_z"]].rolling(window=44,step=22)
rolling_list = list(rolling)[2:10]

In [ ]:
array = np.array(rolling_list)
array.shape

In [181]:
reshaped_array = array.reshape((8, 44, 1, 4))

In [ ]:
reshaped_array

In [ ]:
X_train = np.random.rand(100, 64, 64)  # 100 samples, 64x64 features
X_train.shape

In [166]:
# Aggregate features into timesteps
timesteps = 10
X_train_reshaped = X_train.reshape((
    X_train.shape[0] // timesteps,
    timesteps,
    *X_train.shape[1:],
))

In [ ]:
# Target -> for each sample // timestep group
y_train = np.random.rand(  # samples // timesteps, 1 output
    X_train_reshaped.shape[0],
    1,
)

# LTSM-CNN
model = Sequential([
    Input((None, 64, 64)),
    TimeDistributed(
        Sequential([
            Conv1D(filters=64, kernel_size=3, activation="relu"),
            MaxPooling1D(pool_size=2),
            Flatten(),
        ]),
    ),
    LSTM(100, activation="relu", return_sequences=True),
    LSTM(100, activation="relu"),
    Dense(1),
])
model.compile(optimizer="adam", loss="mse")
model.summary()

# Train
history = model.fit(
    X_train_reshaped,
    y_train,
    epochs=2,
    batch_size=32,
)

# Results
print(history.history)
